In [118]:
from groq import Groq
import base64
from IPython.display import Image
import os
from dotenv import load_dotenv, dotenv_values

In [122]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY)

In [123]:
llava_model = "llava-v1.5-7b-4096-preview"
llama_model = "llama3-groq-8b-8192-tool-use-preview"

In [125]:
def encode_to_64(image_path):
    with open(image_path, 'rb') as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [126]:
def image_to_text(client, model, base64_image, prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model=model
    )
    return chat_completion.choices[0].message.content

In [127]:
def further_query(client, image_description, user_prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """
                    You are an image description chatbot.
                    A short description of an image will be provided to you.
                    Answer the questions asked, based on that description.
                    If the description does not provide any context regarding the query asked, say that 'I could not extract this detail from the image'.
                """
            },
            {
                "role": "user",
                "content": f"{image_description}\n\n{user_prompt}"
            }
        ],
        model=llama_model
    )
    return chat_completion.choices[0].message.content

In [132]:
def final_func(client, image_path, model, user_prompt):
    base64_image = encode_to_64(image_path)
    prompt = "Describe the image"
    image_description = image_to_text(client, model, base64_image, prompt)
    return further_query(client, image_description, user_prompt)



my_finalprompt = "What do you think the people in the image are doing?"
print(final_func(client, "images/image6.jpg", llava_model, my_finalprompt))

Based on the image description, it appears that the group of students are engaged in a discussion or presentation related to coding, possibly with the use of technology. One individual is holding up a book, which might be used to illustrate a point or explain a concept. The atmosphere seems lively, with students both sitting and standing, indicating active participation and engagement.
